In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import cv2
import urllib

from IPython.display import Image

from tqdm.notebook import tqdm_notebook

In [2]:
raw_df = pd.read_csv('./data/raw_data.csv')
raw_df.head()

,id,name,brand_id,brand_name,parent_category_id,parent_category_name,category_id,category_id.1,price,image_url
0,2926,사슴 배냇저고리 세트_Cotton,5,호호당,2.0,의류,25.0,신생아의류,42000,https://hohodangstore.co.kr/web/product/medium...
1,2929,아이 백일 선물 세트,5,호호당,1.0,용품,27.0,기타용품,92000,https://hohodangstore.co.kr/web/product/medium...
2,2930,[카드] 연하장 : New year 2021,5,호호당,1.0,용품,27.0,기타용품,2500,https://hohodangstore.co.kr/web/product/medium...
3,2931,[카드] 일상 : Thanks and love,5,호호당,1.0,용품,27.0,기타용품,2500,https://hohodangstore.co.kr/web/product/medium...
4,2932,금줄,5,호호당,1.0,용품,27.0,기타용품,35000,https://hohodangstore.co.kr/web/product/medium...


In [ ]:
# # only clothes in category15
# # crop_resize

# def crop_image(img, width, height):
#     dst = img
#     if width > height:
#         print('cut width')
#         target = (width-height) // 2
#         dst =  img[:height, target: width-target].copy()
#     elif width < height:
#         print('cut height')
#         target = (height-width) // 2
#         dst = img[target:height-target, :width].copy()
#     return dst

# def convert_image(url):
#     req = urllib.request.urlopen(url)
#     arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
#     img = cv2.imdecode(arr, cv2.IMREAD_COLOR)
        
#     height = img.shape[0]
#     width = img.shape[1]

#     # crop image
#     img = crop_image(img, width, height)

#     dst = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)

#     return dst

# def padding_image(url):
#     req = urllib.request.urlopen(url)
#     arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
#     img = cv2.imdecode(arr, cv2.IMREAD_COLOR)
#     back_color = tuple([int(x) for x in img[1,1]])
#     h, w = img.shape[:2]
#     diff = abs(h - w) // 2

#     if h > w:
#         h_cut = np.float32([[224/ h, 0, diff*224/h], [0, 224/h, 0]])
#         dst = cv2.warpAffine(img, h_cut, (224, 224), None, cv2.INTER_AREA, borderValue=back_color)
#     else:
#         w_cut = np.float32([[224/w, 0, 0], [0, 224/w, diff*225/w]])
#         dst = cv2.warpAffine(img, w_cut, (224,224), None, cv2.INTER_AREA, borderValue=back_color)

#     return dst

# success= 0
# fail = 0

# image_url_list = list(raw_df[raw_df['category_id'] == 15]['image_url'])
        
# size = len(image_url_list)
           
# for j, url in tqdm_notebook(enumerate(image_url_list)):
#     format = url.split('/')[-1].split('.')[-1]
#     try:
#         crop_resize = convert_image(url)
#         padding_img = padding_image(url)
#         #img save
#         cv2.imwrite(f'C:/Users/sanghui/Desktop/toyproject/DL_Project_5/hsh/data/2_15_crop/{j}.{format}', crop_resize)
#         print('crop success')
#         success += 1
#     except:
#         fail += 1
        
#     try:
#         cv2.imwrite(f'C:/Users/sanghui/Desktop/toyproject/DL_Project_5/hsh/data/2_15_padding/{j}.{format}', padding_img)
#         print('padding success')
#         success += 1
#     except:
#         fail += 1

# print(success, fail)


In [4]:
# only clothes
# crop_resize

def crop_image(img, width, height):
    dst = img
    if width > height:
        print('cut width')
        target = (width-height) // 2
        dst =  img[:height, target: width-target].copy()
    elif width < height:
        print('cut height')
        target = (height-width) // 2
        dst = img[target:height-target, :width].copy()
    return dst

def convert_image(url):
    req = urllib.request.urlopen(url)
    arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
    img = cv2.imdecode(arr, cv2.IMREAD_COLOR)
        
    height = img.shape[0]
    width = img.shape[1]

    # crop image
    img = crop_image(img, width, height)

    dst = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)

    return dst

def padding_image(url):
    req = urllib.request.urlopen(url)
    arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
    img = cv2.imdecode(arr, cv2.IMREAD_COLOR)
    back_color = tuple([int(x) for x in img[1,1]])
    h, w = img.shape[:2]
    diff = abs(h - w) // 2

    if h > w:
        h_cut = np.float32([[224/ h, 0, diff*224/h], [0, 224/h, 0]])
        dst = cv2.warpAffine(img, h_cut, (224, 224), None, cv2.INTER_AREA, borderValue=back_color)
    else:
        w_cut = np.float32([[224/w, 0, 0], [0, 224/w, diff*225/w]])
        dst = cv2.warpAffine(img, w_cut, (224,224), None, cv2.INTER_AREA, borderValue=back_color)

    return dst

success= 0
fail = 0

child_category = list(raw_df[raw_df['parent_category_id'] == 2]['category_id'].unique())

for id in tqdm_notebook(child_category):
    image_url_list = list(raw_df[raw_df['category_id'] == id]['image_url'])
        
    size = len(image_url_list)
        
    # random shuffling
    if size >= 3350:
        rand_choices = np.random.choice(image_url_list, 650, replace=False) #비복원 추출
    else:
        rand_choices = np.random.choice(image_url_list, int(size*0.2), replace=False) #비복원 추출
    
    for j, url in enumerate(rand_choices):
        format = url.split('/')[-1].split('.')[-1]
        try:
            crop_resize = convert_image(url)
            padding_img = padding_image(url)
            #img save
            cv2.imwrite(f'C:/Users/sanghui/Desktop/toyproject/DL_Project_5/hsh/data/crop_resize/test/2/{int(id)}/{j}.{format}', crop_resize)
            print('crop success')
            success += 1
        except:
            fail += 1
        
        try:
            cv2.imwrite(f'C:/Users/sanghui/Desktop/toyproject/DL_Project_5/hsh/data/padding_resize/test/2/{int(id)}/{j}.{format}', padding_img)
            print('padding success')
            success += 1
        except:
            fail += 1

print(success, fail)


  0%|          | 0/13 [00:00<?, ?it/s]

crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
cut height
crop success
padding success
cut height
crop success
padding success
crop success
padding success
cut height
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop success
padding success
crop succe